In [6]:
import requests
import warnings
from tqdm import tqdm, trange
import pandas as pd 
import streamlit as st 
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly_express  as px
import plotly.graph_objects as go
import random
import pickle
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [44]:
statement = "income-statement"
tickername = "AAPL"
period = "annual"
key = "6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
ticker = yf.Ticker(tickername)
years = 5

In [ ]:
def get_Cash_Flow_Statement_Data(tickernameI, periodI):
    tickernameL = tickernameI
    periodL = periodI
    url3 = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickernameL}?period={periodL}&apikey={key}"
    cf_json = requests.get(url3).json()
    cf_dfL = pd.DataFrame(cf_json)
    cf_dfL = cf_dfL.reindex(index=cf_dfL.index[::-1])
    cf_dfL.reset_index(inplace=True, drop = True)
    cf_dfL.rename(columns = {'date': 'Date'}, inplace = True) #freeCashFlow
    cf_dfL["Date"] = pd.to_datetime(cf_dfL['Date'])
    cf_dfL.drop(['symbol', 'reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'link', 'finalLink'], axis = 1, inplace=True)
    cf_df_feature_listL = cf_dfL.columns.values.tolist()
    return cf_dfL, cf_df_feature_listL
def get_Balance_Sheet_Data(tickernameI, periodI):
    tickernameL = tickernameI
    periodL = periodI
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickernameL}?period={periodL}&apikey={key}"
    fp_json = requests.get(url).json()
    fp_dfL = pd.DataFrame(fp_json)
    fp_dfL = fp_dfL.reindex(index=fp_dfL.index[::-1])
    fp_dfL.reset_index(inplace=True, drop = True)
    fp_dfL.rename(columns = {'date': 'Date'}, inplace = True)
    fp_df_feature_listL = fp_dfL.columns.values.tolist()
    fp_dfL["Date"] = pd.to_datetime(fp_dfL['Date'])
    fp_dfL.drop(['symbol', 'reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'link', 'finalLink'], axis = 1, inplace=True)
    return fp_dfL, fp_df_feature_listL
def get_income_Statement_data(tickernameI,periodI):
    tickernameL = tickernameI
    periodL = periodI
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickernameL}?period={periodL}&apikey={key}"
    is_json = requests.get(url).json()
    is_dfL = pd.DataFrame(is_json)
    is_dfL.drop(['symbol', 'reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'calendarYear', 'period', 'link', 'finalLink'], axis = 1, inplace=True)
    is_df_features_listL = is_dfL.columns.values.tolist()
    is_dfL = is_dfL.reindex(index=is_dfL.index[::-1])
    is_dfL.rename(columns = {'date':'Date'}, inplace  = True)
    is_dfL["Date"] = pd.to_datetime(is_dfL["Date"])
    is_dfL.reset_index(inplace=True, drop = True)
    return is_dfL, is_df_features_listL
def get_stock_price(tickernameI, is_dfI,years):
    is_dfL = is_dfI.iloc[-years:]
    is_dfL = is_dfL.reset_index(drop=True)
    tickernameL = tickernameI
    start_dateL = is_dfL["Date"][0].date().strftime("%Y-%m-%d")
    rows = is_dfL.shape[0] -1
    start_data_offset1 = (is_dfL["Date"][0] + pd.Timedelta(days = 1)).date().strftime("%Y-%m-%d")
    start_data_offset2 = (is_dfL["Date"][0] + pd.Timedelta(days = 2)).date().strftime("%Y-%m-%d")
    start_data_offset3 = (is_dfL["Date"][0] + pd.Timedelta(days = 3)).date().strftime("%Y-%m-%d")
    start_data_offset4 = (is_dfL["Date"][0] + pd.Timedelta(days = 4)).date().strftime("%Y-%m-%d")
    end_date = is_dfL["Date"][rows].date().strftime("%Y-%m-%d")
    end_date_offset1 = (is_dfL["Date"][rows] + pd.Timedelta(days = 1)).date().strftime("%Y-%m-%d")
    end_date_offset2 = (is_dfL["Date"][rows] + pd.Timedelta(days = 2)).date().strftime("%Y-%m-%d")
    end_date_offset3 = (is_dfL["Date"][rows] + pd.Timedelta(days = 3)).date().strftime("%Y-%m-%d")
    end_date_offset4 = (is_dfL["Date"][rows] + pd.Timedelta(days = 4)).date().strftime("%Y-%m-%d")
    now = datetime.today().strftime('%Y-%m-%d')
    price_dfL = pd.DataFrame(yf.download(tickers=tickernameL,start = "2017-01-01" , end =now, interval="1d")["Close"])
    price_dfL.reset_index(inplace=True)
    if(price_dfL[price_dfL["Date"] == start_dateL].count()[0] == 0): #no price for start date use first price
        start_priceL = price_dfL.iloc[0,1]
    else:
        start_priceL = price_dfL.query(f"Date == '{start_dateL}' | Date == '{start_data_offset1}' | Date == '{start_data_offset2}'| Date == '{start_data_offset3}'| Date == '{start_data_offset4}' ").iloc[0,1]
    end_price = price_dfL.query(f"Date == '{end_date}' | Date == '{end_date_offset1}' | Date == '{end_date_offset2}' | Date == '{end_date_offset3}' | Date == '{end_date_offset4}' ").iloc[0,1]
    price_cagrL = round(((end_price/start_priceL)**(1/is_dfL.shape[0])-1)*100,2)
    return price_dfL, price_cagrL, start_dateL
def process_revenue_metrics(is_dfI, periodI,is_df_features_listI, years ):
    is_dfL = is_dfI.iloc[-years:]
    is_dfL.reset_index(inplace = True, drop = True)
    is_df_features_listL = is_df_features_listI
    rows = is_dfL.shape[0]-1
    num_of_periods = is_dfL.shape[0]
    period_name = periodI
    cagr_list = []
    total_growth_list = []
    percent_of_revenue_list = []
    for i in range(0,len(is_df_features_listL)):
        #CAGR
        if (i == 0):     
            cagr_list.append(float("nan"))
            total_growth_list.append(float("nan"))
            percent_of_revenue_list.append(float("nan"))
            continue
        end = is_dfL.iloc[rows,i]
        start = is_dfL.iloc[0,i]
        if start == 0:
            cagr = float("nan")
            cagr_list.append(cagr)
        else:
            cagr = ((end/start)**(1/num_of_periods)-1).real
            cagr_list.append(round(cagr*100,2))
        #Total Growth
        if start == 0:
            total_growth = float("nan")
            total_growth_list.append(total_growth)
        else:
            total_growth = round(((end-start)/start)*100,2)
            total_growth_list.append(total_growth)
        #Percent of Revenue
        revenue = is_dfL.loc[rows,"revenue"]
        percent_of_revenue = abs(round(((end/revenue)*100),2))
        percent_of_revenue_list.append(percent_of_revenue)
    id_df_summary_dfL = pd.DataFrame()
    id_df_summary_dfL["Feature"] = is_df_features_listL
    id_df_summary_dfL[f"{period_name} Compounded Growth Rate %"] = cagr_list
    id_df_summary_dfL["Total Growth %"] = total_growth_list
    id_df_summary_dfL["Percentage Of Revenue(Last)"] = percent_of_revenue_list
    return id_df_summary_dfL, is_dfL
def process_historic_multiples(price_dfI, is_dfI, years ):
    price_dfL = price_dfI
    is_dfL = is_dfI.iloc[-years:]
    Multiples_dfL = pd.DataFrame()
    Multiples_dfL["Date"] = is_dfL["Date"]
    Multiples_dfL["revenue"] = is_dfL["revenue"]
    Multiples_dfL["weightedAverageShsOutDil"] = is_dfL["weightedAverageShsOutDil"]
    Multiples_dfL["Basic RPS"] = Multiples_dfL["revenue"]/Multiples_dfL["weightedAverageShsOutDil"]
    Multiples_dfL["epsdiluted"] = is_dfL['epsdiluted']
    price_dfL["EPS"] = float("nan")
    price_dfL["RPS"] = float("nan")
    for i in Multiples_dfL.iterrows():
        rps = i[1][3]
        eps = i[1][4]
        date = i[1][0]
        dateoffset1 = date + pd.Timedelta(days = 1)
        dateoffset2 = date + pd.Timedelta(days = 2)
        dateoffset3 = date + pd.Timedelta(days = 3)
        dateoffset4 = date + pd.Timedelta(days = 4)
        price_dfL.loc[price_dfL.query(f"Date == '{date}' | Date == '{dateoffset1}' | Date == '{dateoffset2}'| Date == '{dateoffset3}' | Date == '{dateoffset4}'").index,"EPS"] = eps
        price_dfL.loc[price_dfL.query(f"Date == '{date}' | Date == '{dateoffset1}' | Date == '{dateoffset2}'| Date == '{dateoffset3}' | Date == '{dateoffset4}'").index,"RPS"] = rps
    price_features_listL = price_dfL.columns.values.tolist()
    price_dfL = expand_numbers(price_dfL, price_features_listL)
    price_dfL["PE"] = price_dfL["Close"]/price_dfL["EPS"]
    price_dfL["PS"] = price_dfL["Close"]/price_dfL["RPS"]
    price_dfL["Volatility"] = price_dfL["Close"].pct_change()
    return price_dfL, Multiples_dfL
def plot_revenue_metrics(id_df_summary_dfI,periodI):
    id_df_summary_dfL = id_df_summary_dfI
    periodL = periodI
    metric_choices = [f"{periodL} Compounded Growth Rate %", "Total Growth %", "Percentage Of Revenue(Last)"]
    features_list = id_df_summary_dfL["Feature"].unique().tolist()
    with st.form("my_form"):
        metric = st.selectbox("Choose Metric", options=metric_choices)
        features_chosen = st.multiselect("Choose Feature", options=features_list)
        rev_metr_submit = st.form_submit_button("Confirm")
    
    if rev_metr_submit | st.session_state['revenue_metrics'] == True:
        st.session_state['revenue_metrics'] = True
        cagr_plot = go.Figure()
        for feature in features_chosen:
            feature_index = features_list.index(feature)
            cagr_plot.add_trace(go.Bar(x = [feature], y = [id_df_summary_dfL.loc[feature_index, metric]], name = f"{feature}"))
        
        return cagr_plot
def stock_analysis():
    stock_list_pd = pd.read_pickle("StockList")
    if 'revenue_metrics' not in st.session_state:
        st.session_state.revenue_metrics = False
    if 'historic_multiples' not in st.session_state:
        st.session_state.historic_multiples = False
    if 'freegraph1' not in st.session_state:
        st.session_state.freegraph1 = False
    if 'stock_submission' not in st.session_state:
        st.session_state.stock_submission = False
    
    
    st.markdown('# Enter Stock Name')
    tickername = st.selectbox("Input Stock Ticker", options=stock_list_pd["symbol"].to_list(), index = stock_list_pd["symbol"].to_list().index("AAPL") )
    period = st.selectbox("Select Period", options=["annual", "quarterly"])
    years_of_data = st.number_input("Number of Reports", step =1, min_value = 2, )
    
    
    is_df,is_df_features_list = get_income_Statement_data(tickername, period)
    price_df,price_cagr, start_date= get_stock_price(tickername, is_df, years_of_data)
    fp_df, fp_df_feature_list = get_Balance_Sheet_Data(tickername, period)
    cf_df, cf_df_feature_list = get_Cash_Flow_Statement_Data(tickername, period)
    description_df, description_features_list = get_description_data(tickername)
    id_df_summary_df, is_df = process_revenue_metrics(is_df, period, is_df_features_list,years_of_data)
    s = process_company_description(description_df )
    price_df, Multiples_df = process_historic_multiples(price_df, is_df, years_of_data)
    
    with st.expander("Price"):
        st.markdown('# Price')
        st.plotly_chart(plot_stock_price(price_df, price_cagr), use_container_width=True)
    
    with st.expander("Description"):
        st.markdown(plot_description(s))
        st.markdown('---')
    
    with st.expander("Profitability Metrics"):
        st.markdown('# Profitability Metrics')
        st.plotly_chart(plot_revenue_metrics(id_df_summary_df,period), use_container_width=True)
    
    with st.expander("Historic Multiples"):
        st.markdown('# Historic Multiples')
        st.plotly_chart(plot_historic_multiples(price_df,start_date), use_container_width=True)
        st.markdown('---')
    
    with st.expander("Free Plot 1"):
        st.markdown('# Free Plot')
        st.plotly_chart(plot_free_graph1(is_df,is_df_features_list,fp_df,fp_df_feature_list,cf_df,cf_df_feature_list,years_of_data))
        st.markdown('---')
    
    usa_stock_list = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")['symbol'].to_list()
    if tickername in usa_stock_list:
        with st.expander("Revenue Breakdown"):
            st.markdown("# Revenue Breakdown")
            rev_seg_df = get_rev_seg_data(tickername,period )
            
            st.plotly_chart(plot_rev_segmentation_snapshot(rev_seg_df), use_container_width=True)
            st.plotly_chart(plot_rev_seg_over_time(rev_seg_df), use_container_width=True)
            st.markdown('---')


In [7]:
def expand_numbers(df, list_of_columns):
    for name_of_column in list_of_columns:
        most_recent_value = 0
        for row in df.index:
            if pd.isna(df.loc[row, name_of_column]) and most_recent_value == 0:
                continue
            elif pd.isna(df.loc[row, name_of_column]):
                df.loc[row, name_of_column] = most_recent_value
            else:
                most_recent_value = df.loc[row, name_of_column]
    return df


In [154]:
response = requests.get("https://financialmodelingprep.com/api/v3/stock-screener?apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI")
df = pd.DataFrame(response.json())
stock_df = df.query("isEtf == False & isFund == False ")
stock_df = stock_df.sort_values(by = 'marketCap', ascending=False)
stock_df["Sector"] = stock_df["sector"]
stock_df["Industry"] = stock_df["industry"]
stock_df = stock_df.drop(['exchange','exchangeShortName', 'isEtf', 'isFund', 'isActivelyTrading','sector','industry'  ], axis=1)
stock_df.reset_index(inplace=True, drop=True)
stock_df

,symbol,companyName,marketCap,beta,price,lastAnnualDividend,volume,country,Sector,Industry
0,MSFT,Microsoft Corporation,3161800828800,0.877,425.52,3.00,16471297,US,Technology,Software - Infrastructure
1,AAPL,Apple Inc.,2618637402000,1.276,169.58,0.96,41844023,US,Technology,Consumer Electronics
2,NVDA,NVIDIA Corporation,2200200000000,1.744,880.08,0.16,39737044,US,Technology,Semiconductors
3,AMZN,"Amazon.com, Inc.",1922396118000,1.160,185.07,0.00,41959426,US,Consumer Cyclical,Specialty Retail
4,GOOGL,Alphabet Inc.,1904198753763,1.044,152.50,0.00,23278240,US,Communication Services,Internet Content & Information
...,...,...,...,...,...,...,...,...,...,...
605,DOV,Dover Corporation,24156195840,1.274,175.84,2.04,788308,US,Industrials,Industrial - Machinery
606,TW,Tradeweb Markets Inc.,24140811571,0.922,102.40,0.37,875888,US,Financial Services,Financial - Capital Markets
607,FITB,Fifth Third Bancorp,24129315000,1.271,35.30,1.38,2888348,US,Financial Services,Banks - Regional
608,IX,ORIX Corporation,24077406230,0.903,105.50,2.96,8070,JP,Financial Services,Financial - Credit Services


In [887]:
def get_stock_data(tickerI):
    tickerL = tickerI
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_is_df = pd.DataFrame(response.json())
    if annual_is_df.shape == (0, 0):
        annual_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        last_date_annual = annual_is_df.loc[0,'date']
        annual_is_df = annual_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_is_df = annual_is_df.add_suffix(f'(FY)')
        
    
    
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_is_df = pd.DataFrame(response.json())
    if quarterly_is_df.shape == (0, 0):
        quarterly_is_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        last_date_quarterly = quarterly_is_df.loc[0,'date']
        quarterly_is_df = quarterly_is_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_is_df = quarterly_is_df.add_suffix(f'(FQ)')
        
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_fp_df = pd.DataFrame(response.json())
    if annual_fp_df.shape == (0, 0):
        annual_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_fp_df = annual_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_fp_df = annual_fp_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_fp_df = pd.DataFrame(response.json())
    if quarterly_fp_df.shape == (0, 0):
        quarterly_fp_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_fp_df = quarterly_fp_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_fp_df = quarterly_fp_df.add_suffix(f'(FQ)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=annual&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    annual_cf_df = pd.DataFrame(response.json())
    if annual_cf_df.shape == (0, 0):
        annual_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        annual_cf_df = annual_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        annual_cf_df = annual_cf_df.add_suffix(f'(FY)')
    
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{tickerL}?period=quarterly&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI"
    response = requests.get(url)
    quarterly_cf_df = pd.DataFrame(response.json())
    if quarterly_cf_df.shape == (0, 0):
        quarterly_cf_df =  pd.DataFrame((float('Nan'), float('Nan')))
    else:
        quarterly_cf_df = quarterly_cf_df.drop(['date','symbol','reportedCurrency', 'cik', 'fillingDate', 'acceptedDate', 'link', 'finalLink'], axis=1)
        quarterly_cf_df = quarterly_cf_df.add_suffix(f'(FQ)')
    
    stock_data = pd.concat([annual_is_df, quarterly_is_df, annual_fp_df, quarterly_fp_df, annual_cf_df  ], axis = 1)
    stock_data = pd.concat([stock_data,quarterly_cf_df], axis = 1)
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='calendarYear')))]
    stock_data = stock_data[stock_data.columns.drop(list(stock_data.filter(regex='period')))]
    stock_data = stock_data.loc[:,~stock_data.columns.duplicated()].copy()
    stock_data1 = stock_data.iloc[0:9,]
    
    if quarterly_cf_df.shape[0]<9:
        for year in range(1,quarterly_cf_df.shape[0]):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    else:
        for year in range(1,9):
            stock_data1 = pd.concat([stock_data1, pd.DataFrame(stock_data1.iloc[year,0:198].add_suffix(f'-{year}')).transpose().reset_index(drop=True)], axis=1)
    stock_data1 = stock_data1.iloc[0,:]
    stock_data_final = pd.DataFrame(stock_data1).transpose()
    stock_data_final.insert(loc = 0, column ="Ticker", value=f"{tickerL}" )
    stock_data_final.insert(loc = 0, column ="Date(FY)", value=f"{last_date_annual}" )
    stock_data_final.insert(loc = 0, column ="Date(FQ)", value=f"{last_date_quarterly}" )
    
    return stock_data_final

In [974]:
def build_database():
    us_stock_database = pd.DataFrame()
    stock_list_pd = pd.read_pickle("StockList")
    stock_list_pd = stock_list_pd.query("type == 'stock'")
    stock_list_pd = stock_list_pd.query("exchangeShortName == 'NYSE' | exchangeShortName == 'NASDAQ'")
    stock_list_pd.reset_index(inplace=True, drop=True)
    stock_list = stock_list_pd["symbol"].to_list()
    for ticker in tqdm(["AAPL", "MSFT", "NVDA", "GOOGL"]):
        #tqdm._instances.clear()
        us_stock_database = pd.concat([us_stock_database, get_stock_data(ticker)])
        #print(f'processed {ticker}')
        us_stock_database.reset_index(drop=True)
    return us_stock_database


In [978]:
def unpack_annual_data_with_Date(tickernameI, dbI):
    raw_data = dbI.query(f"Ticker == '{tickernameI}'")
    last_annual_date = raw_data.loc[0,'Date(FY)']
    raw_data = raw_data.drop(['Ticker', 'Date(FQ)', 'Date(FY)'], axis=1)
    raw_data = raw_data.transpose()
    raw_data = raw_data.reset_index()
    raw_data.rename(columns = {'index':'feature', 0 : 'value'}, inplace=True)
    raw_data.dropna(inplace=True)
    annual_df = raw_data.query("feature.str.contains('FY')")
    annual_stock_data = pd.DataFrame()
    current = annual_df.query(f'feature.str.endswith(")")')
    current = current.reset_index(drop=True)
    annual_stock_data["feature"] = current['feature']
    annual_stock_data['Current'] = current['value']
    annual_stock_data
    for year in range(1,9):
        temp = annual_df.query(f'feature.str.endswith("{year}")')
        temp.reset_index(inplace=True, drop=True)
        annual_stock_data[f"-{year}"] = temp[f"value"]
    annual_stock_data.set_index('feature', inplace=True)
    annual_stock_data.rename(columns = {'Current' : '0'}, inplace=True)
    annual_stock_data = annual_stock_data.transpose()
    annual_stock_data = annual_stock_data.reindex(index=annual_stock_data.index[::-1])
    annual_stock_data_features_list  = annual_stock_data.columns.to_list()
    date_list = []
    for year in annual_stock_data.index.to_list():
        yearsback = abs(int(year))
        date = pd.to_datetime(last_annual_date) - relativedelta(years=yearsback)
        date_list.append(date)
    annual_stock_data.insert(loc=0, column='Date', value=date_list)
    annual_stock_data = annual_stock_data.reset_index(drop=True)
    return annual_stock_data, annual_stock_data_features_list

In [1059]:
db = pd.read_pickle('sample_db.pkl')
annual_stock_data, annual_stock_data_features_list = unpack_annual_data_with_Date(tickername, db)

In [1060]:
annual_stock_data

feature,Date,revenue(FY),costOfRevenue(FY),grossProfit(FY),grossProfitRatio(FY),researchAndDevelopmentExpenses(FY),generalAndAdministrativeExpenses(FY),sellingAndMarketingExpenses(FY),sellingGeneralAndAdministrativeExpenses(FY),otherExpenses(FY),...,dividendsPaid(FY),otherFinancingActivites(FY),netCashUsedProvidedByFinancingActivities(FY),effectOfForexChangesOnCash(FY),netChangeInCash(FY),cashAtEndOfPeriod(FY),cashAtBeginningOfPeriod(FY),operatingCashFlow(FY),capitalExpenditure(FY),freeCashFlow(FY)
0,2015-09-30,2.337150e+11,1.400890e+11,9.362600e+10,0.400599,8.067000e+09,0.0,0.0,1.432900e+10,1.285000e+09,...,-1.156100e+10,2.855500e+10,-1.771600e+10,0.0,7.276000e+09,2.112000e+10,1.384400e+10,8.126600e+10,-1.148800e+10,6.977800e+10
1,2016-09-30,2.156390e+11,1.313760e+11,8.426300e+10,0.390760,1.004500e+10,0.0,0.0,1.419400e+10,1.348000e+09,...,-1.215000e+10,2.339400e+10,-2.048300e+10,0.0,-6.360000e+08,2.048400e+10,2.112000e+10,6.582400e+10,-1.354800e+10,5.227600e+10
2,2017-09-30,2.292340e+11,1.410480e+11,8.818600e+10,0.384699,1.158100e+10,0.0,0.0,1.526100e+10,2.745000e+09,...,-1.276900e+10,3.126700e+10,-1.734700e+10,0.0,-1.950000e+08,2.028900e+10,2.048400e+10,6.359800e+10,-1.279500e+10,5.080300e+10
3,2018-09-30,2.655950e+11,1.637560e+11,1.018390e+11,0.383437,1.423600e+10,0.0,0.0,1.670500e+10,2.005000e+09,...,-1.371200e+10,4.405000e+09,-8.787600e+10,0.0,5.624000e+09,2.591300e+10,2.028900e+10,7.743400e+10,-1.331300e+10,6.412100e+10
4,2019-09-30,2.601740e+11,1.617820e+11,9.839200e+10,0.378178,1.621700e+10,0.0,0.0,1.824500e+10,1.807000e+09,...,-1.411900e+10,-1.936000e+09,-9.097600e+10,0.0,2.431100e+10,5.022400e+10,2.591300e+10,6.939100e+10,-1.049500e+10,5.889600e+10
5,2020-09-30,2.745150e+11,1.695590e+11,1.049560e+11,0.382332,1.875200e+10,0.0,0.0,1.991600e+10,8.030000e+08,...,-1.408100e+10,1.233100e+10,-8.682000e+10,0.0,-1.043500e+10,3.978900e+10,5.022400e+10,8.067400e+10,-7.309000e+09,7.336500e+10
6,2021-09-30,3.658170e+11,2.129810e+11,1.528360e+11,0.417794,2.191400e+10,0.0,0.0,2.197300e+10,2.580000e+08,...,-1.446700e+10,1.473000e+10,-9.335300e+10,0.0,-3.860000e+09,3.592900e+10,3.978900e+10,1.040380e+11,-1.108500e+10,9.295300e+10
7,2022-09-30,3.943280e+11,2.235460e+11,1.707820e+11,0.433096,2.625100e+10,0.0,0.0,2.509400e+10,-3.340000e+08,...,-1.484100e+10,3.037000e+09,-1.107490e+11,0.0,-1.095200e+10,2.497700e+10,3.592900e+10,1.221510e+11,-1.070800e+10,1.114430e+11
8,2023-09-30,3.832850e+11,2.141370e+11,1.691480e+11,0.441311,2.991500e+10,0.0,0.0,2.493200e+10,-5.650000e+08,...,-1.502500e+10,-6.012000e+09,-1.084880e+11,0.0,5.760000e+09,3.073700e+10,2.497700e+10,1.105430e+11,-1.095900e+10,9.958400e+10


In [ ]:
def process_revenue_metrics_db(annual_stock_dataI, years, periodI):
    is_dfL = annual_stock_dataI
    
    is_dfL = annual_stock_dataI.iloc[-years:]
    is_dfL.reset_index(inplace = True, drop = True)
    is_dfL = is_dfL.drop(['Date'], axis = 1)
    is_df_features_listL =  is_dfL.columns.to_list()
    rows = is_dfL.shape[0]-1
    num_of_periods = is_dfL.shape[0]
    period = periodI

    cagr_list = []
    total_growth_list = []
    percent_of_revenue_list = []
    if period == 'annual':
        period_name = 'FY'
    if period == 'quarterly':
        period_name = 'FQ'
    for i in range(0,len(is_df_features_listL)):
        #CAGR
        
        end = is_dfL.iloc[rows,i]
        start = is_dfL.iloc[0,i]
        
        if start == 0:
            cagr = float("nan")
            cagr_list.append(cagr)
        else:
            cagr = ((end/start)**(1/num_of_periods)-1).real
            cagr_list.append(round(cagr*100,2))
        #Total Growth
        if start == 0:
            total_growth = float("nan")
            total_growth_list.append(total_growth)
        else:
            total_growth = round(((end-start)/start)*100,2)
            total_growth_list.append(total_growth)
        #Percent of Revenue
        revenue = is_dfL.loc[rows,f"revenue({period_name})"]
        percent_of_revenue = abs(round(((end/revenue)*100),2))
        percent_of_revenue_list.append(percent_of_revenue)
    id_df_summary_dfL = pd.DataFrame()
    id_df_summary_dfL["Feature"] = is_df_features_listL
    id_df_summary_dfL[f"{period_name} Compounded Growth Rate %"] = cagr_list
    id_df_summary_dfL["Total Growth %"] = total_growth_list
    id_df_summary_dfL["Percentage Of Revenue(Last)"] = percent_of_revenue_list
    return id_df_summary_dfL, is_dfL

In [ ]:
process_revenue_metrics_db(annual_stock_data, )

In [990]:
start = annual_stock_data.iloc[0,'netIncome(FY)']

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [988]:
annual_stock_data_features_list

['revenue(FY)',
 'costOfRevenue(FY)',
 'grossProfit(FY)',
 'grossProfitRatio(FY)',
 'researchAndDevelopmentExpenses(FY)',
 'generalAndAdministrativeExpenses(FY)',
 'sellingAndMarketingExpenses(FY)',
 'sellingGeneralAndAdministrativeExpenses(FY)',
 'otherExpenses(FY)',
 'operatingExpenses(FY)',
 'costAndExpenses(FY)',
 'interestIncome(FY)',
 'interestExpense(FY)',
 'depreciationAndAmortization(FY)',
 'ebitda(FY)',
 'ebitdaratio(FY)',
 'operatingIncome(FY)',
 'operatingIncomeRatio(FY)',
 'totalOtherIncomeExpensesNet(FY)',
 'incomeBeforeTax(FY)',
 'incomeBeforeTaxRatio(FY)',
 'incomeTaxExpense(FY)',
 'netIncome(FY)',
 'netIncomeRatio(FY)',
 'eps(FY)',
 'epsdiluted(FY)',
 'weightedAverageShsOut(FY)',
 'weightedAverageShsOutDil(FY)',
 'cashAndCashEquivalents(FY)',
 'shortTermInvestments(FY)',
 'cashAndShortTermInvestments(FY)',
 'netReceivables(FY)',
 'inventory(FY)',
 'otherCurrentAssets(FY)',
 'totalCurrentAssets(FY)',
 'propertyPlantEquipmentNet(FY)',
 'goodwill(FY)',
 'intangibleAssets(

In [86]:
now = datetime.now().strftime("%Y-%m-%d")
three_days = (datetime.now() + pd.Timedelta(days = 3)).strftime("%Y-%m-%d")
response = requests.get(f'https://financialmodelingprep.com/api/v3/earning_calendar?from={now}&to={three_days}&apikey=6ulfs8VItWZcKZTMzNJxwmikpQvSF1cI')
raw_upcoming_earnings_df = pd.DataFrame(response.json())
upcoming_earnings_df = pd.DataFrame()
upcoming_earnings_df["Earnings Date"] = raw_upcoming_earnings_df["updatedFromDate"]
upcoming_earnings_df["Ticker"] = raw_upcoming_earnings_df["symbol"]
upcoming_earnings_df.head(30)

,Earnings Date,Ticker
0,2024-04-07,ASTERDM.NS
1,2024-04-06,HZM.L
2,2024-04-07,QBIO
3,2024-04-07,SQIDF
4,2024-04-07,PANACEABIO.NS
5,2024-04-07,BGO.L
6,2024-04-06,POLB.L
7,2024-04-07,600699.SS
8,2024-04-07,VIVE
9,2024-04-07,VIVXF
